# Описание кейса

Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

## Шаг 1. Подключим нужные библиотеки и считаем данные

In [1]:
import pandas as pd

In [32]:
df = pd.read_csv('KC_case_data.csv')

In [3]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


## Шаг 2. Сделаем первичный осмотр данных и предобработку

In [4]:
# Функция первичного обзора данных
def review(data):
    display(data.head(5))
    print()
    print(data.info())
    print()
    print('Пропуски:', data.isna().sum())
    print()
    print('Явные дубликаты:')
    if data.duplicated().sum() > 0:
        print(data.duplicated().sum())
    else:
        print('Не найдено')
    print()
    print('Размер датафрейма:', data.shape)

In [5]:
review(df)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747968 entries, 0 to 2747967
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   date          object 
 1   event         object 
 2   purchase_sum  float64
 3   os_name       object 
 4   device_id     int64  
 5   gender        object 
 6   city          object 
 7   utm_source    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 167.7+ MB
None

Пропуски: date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

Явные дубликаты:
Не найдено

Размер датафрейма: (2747968, 8)


## Вывод
1) Пропуски в purchase_sum логичны, так как эти поля должны быть только тогда, когда event равен "purchased" 

2) Преобразуем в нужный тип данных дату

In [25]:
df['date'] = pd.to_datetime(df['date'])

# Выполнение заданий

## Задание 1.
Посчитать MAU февраля 2020 года

In [7]:
# Фильтруем данные за февраль
february_data = df[(df['date'] >= '2020-02-01') & (df['date'] <= '2020-02-29')]

# Подсчитываем уникальных пользователей
mau_february = february_data['device_id'].nunique()
print(f"MAU февраля: {mau_february}")


MAU февраля: 75032


## Задание 2.
Узнать количество установок в январе 2020 года

In [8]:
# Фильтруем данные за январь
january_installations = df[(df['date'] >= '2020-01-01') & (df['date'] <= '2020-01-31') & (df['event'] == 'app_install')]

# Подсчитываем уникальные установки
installations_january = january_installations['device_id'].nunique()
print(f"Количество установок в январе: {installations_january}")


Количество установок в январе: 80297


## Задание 3. 
Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей?

### Шаг 1. Фильтрация событий установки и покупки

In [9]:
# Фильтруем только события установки и покупки
installs = df[df['event'] == 'app_install'].copy()
purchases = df[df['event'] == 'purchase'].copy()

In [10]:
# Убираем дубли и оставляем только уникальные даты
installs = installs[['device_id', 'date']].drop_duplicates()
purchases = purchases[['device_id', 'date', 'purchase_sum']].drop_duplicates()

### Шаг 2.  Определение первой установки и первой покупки для каждого пользователя

In [11]:
# Определяем дату первой установки для каждого пользователя
first_installs = installs.groupby('device_id')['date'].min().reset_index()
first_installs.rename(columns={'date': 'install_date'}, inplace=True)

# Определяем дату первой покупки для каждого пользователя
first_purchases = purchases.groupby('device_id')['date'].min().reset_index()
first_purchases.rename(columns={'date': 'purchase_date'}, inplace=True)


### Шаг 3. Объединение данных об установках и покупках

In [12]:
# Объединяем данные об установках и покупках по device_id
user_data = pd.merge(first_installs, first_purchases, on='device_id', how='left')

# Преобразуем столбцы с датами в формат datetime
user_data['install_date'] = pd.to_datetime(user_data['install_date'])
user_data['purchase_date'] = pd.to_datetime(user_data['purchase_date'])


### Шаг 4. Вычисление времени от установки до покупки и фильтрация по 7 дням

In [13]:
# Вычисляем количество дней между установкой и первой покупкой
user_data['days_since_install'] = (user_data['purchase_date'] - user_data['install_date']).dt.days

# Фильтруем покупки, произошедшие в течение 7 дней после установки
user_data['converted'] = user_data['days_since_install'].apply(lambda x: x <= 7 if pd.notna(x) else False)


### Шаг 5. Присвоение когорты по дате первой установки

In [14]:
# Присваиваем пользователям когортные группы по дате первой установки
user_data['cohort_date'] = user_data['install_date'].dt.date


### Шаг 6. Подсчет конверсии по когортам

In [15]:
# Группируем данные по дате установки (когорта) и рассчитываем конверсии
cohort_conversion = user_data.groupby('cohort_date').agg(
    installs=('device_id', 'count'),          # Количество установок
    conversions=('converted', 'sum')          # Количество конверсий
).reset_index()

# Рассчитываем конверсию (в процентах)
cohort_conversion['conversion_rate'] = (cohort_conversion['conversions'] / cohort_conversion['installs']) * 100

# Округляем конверсию до 1 знака после запятой
cohort_conversion['conversion_rate'] = cohort_conversion['conversion_rate'].round(1)


In [16]:
cohort_conversion.head(15)

,cohort_date,installs,conversions,conversion_rate
0,2020-01-01,3579,1408,39.3
1,2020-01-02,3144,1186,37.7
2,2020-01-03,2402,834,34.7
3,2020-01-04,1831,639,34.9
4,2020-01-05,1671,587,35.1
5,2020-01-06,1488,487,32.7
6,2020-01-07,1298,423,32.6
7,2020-01-08,1154,386,33.4
8,2020-01-09,1424,558,39.2
9,2020-01-10,13106,4161,31.7


### Ответ: 01.01.2020, конверсия 39.3

## Задание 4
С какого платного маркетингового канала пришло больше всего новых пользователей? 

In [17]:
# Фильтруем только события установки приложения
installs = df[df['event'] == 'app_install'].copy()

# Убираем дубли, чтобы считать каждого пользователя только один раз
installs = installs[['device_id', 'utm_source']].drop_duplicates()

# Фильтруем платные маркетинговые каналы
paid_channels = installs[installs['utm_source'].isin(['yandex-direct', 'google_ads', 'vk_ads', 'instagram_ads', 'facebook_ads', 'referal'])]


In [18]:
paid_channels['utm_source'].value_counts().reset_index()

,index,utm_source
0,yandex-direct,29368
1,google_ads,26286
2,vk_ads,23189
3,instagram_ads,20096
4,facebook_ads,13916
5,referal,9282


### Ответ: Яндекс-директ

## Задание 5

Проанализировать на каком этапе воронки отваливается бОльшая часть клиентов. Посмотреть отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

In [34]:
registration_data = df[df['event'] == 'register'].groupby('device_id')['date'].min().reset_index()

In [36]:
registration_data.rename(columns={'date':'registration_date'}, inplace = True)
df = df.merge(registration_data, on='device_id', how='left')

In [38]:
# Отмечаем зарегистрирован ли пользователь на момент события
df['is_registered'] = df.apply(lambda x: x['date'] >= x['registration_date'] if pd.notnull(x['registration_date']) else False, axis=1)

In [39]:
# Последовательность шагов в воронке
steps = ['app_install', 'app_start', 'register', 'search', 'open_item', 'choose_item', 'tap_basket', 'purchase']

# Функция для расчета количества пользователей на каждом шаге
def funnel_step_counts(data, steps, is_registered):
    step_counts = {}
    for step in steps:
        if step == 'register':
            step_counts[step] = data[(data['event'] == step) & (data['is_registered'] == is_registered)]['device_id'].nunique()
        else:
            step_counts[step] = data[(data['event'] == step) & (data['is_registered'] | (step != 'purchase'))]['device_id'].nunique()
    return step_counts


In [40]:
# Подсчет для зарегистрированных и незарегистрированных пользователей
registered_counts = funnel_step_counts(df, steps, True)
unregistered_counts = funnel_step_counts(df, steps, False)

In [41]:
# Функция для расчета конверсии
def calculate_conversion(step_counts):
    conversions = {}
    prev_count = None
    for step, count in step_counts.items():
        if prev_count is not None:
            conversions[step] = count / prev_count if prev_count > 0 else 0
        prev_count = count
    return conversions

In [42]:
# Вычисление конверсии для зарегистрированных и незарегистрированных пользователей
registered_conversions = calculate_conversion(registered_counts)
unregistered_conversions = calculate_conversion(unregistered_counts)


In [43]:
# Находим шаг с наименьшей конверсией для зарегистрированных пользователей
lowest_conversion_step_registered = min(registered_conversions, key=registered_conversions.get)

# Находим шаг с наименьшей конверсией для незарегистрированных пользователей
lowest_conversion_step_unregistered = min(unregistered_conversions, key=unregistered_conversions.get)

In [44]:
# Результаты
print(f'Шаг с наименьшей конверсией для зарегистрированных пользователей: {lowest_conversion_step_registered}')
print(f'Шаг с наименьшей конверсией для незарегистрированных пользователей: {lowest_conversion_step_unregistered}')


Шаг с наименьшей конверсией для зарегистрированных пользователей: open_item
Шаг с наименьшей конверсией для незарегистрированных пользователей: register


### Ответ: шаг с наименшье конверсией для зарегистрированных пользователей - это вход в корзину

## Задание 6

Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?


In [45]:
# Фильтрация событий первого открытия приложения
first_open = df[df['event'] == 'app_start'].groupby('device_id')['date'].min().reset_index()
first_open = first_open.merge(df, on=['device_id', 'date'])
first_open_users = first_open.groupby('utm_source')['device_id'].nunique().reset_index()
first_open_users.columns = ['utm_source', 'first_open_count']

In [46]:
# Фильтрация событий первой покупки
first_purchase = df[df['event'] == 'purchase'].groupby('device_id')['date'].min().reset_index()
first_purchase = first_purchase.merge(df, on=['device_id', 'date'])
first_purchase_users = first_purchase.groupby('utm_source')['device_id'].nunique().reset_index()
first_purchase_users.columns = ['utm_source', 'first_purchase_count']

In [48]:
# Шаг 3: Расчет коэффициента конверсии
conversion_data = first_open_users.merge(first_purchase_users, on='utm_source', how='left')
conversion_data['first_purchase_count'] = conversion_data['first_purchase_count'].fillna(0)
conversion_data['CR'] = conversion_data['first_purchase_count'] / conversion_data['first_open_count']

In [51]:
conversion_data.sort_values(by='CR')

,utm_source,first_open_count,first_purchase_count,CR
6,yandex-direct,34441,10936,0.317529
2,google_ads,31437,10167,0.323409
0,-,41456,14786,0.356667
3,instagram_ads,24818,9820,0.395681
5,vk_ads,27905,11460,0.410679
1,facebook_ads,18844,7903,0.419391
4,referal,11983,5803,0.484269


### Ответ: и снова яндекс:)

## Задание 7
Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [52]:
# Фильтрация событий purchase
purchase_data = df[df['event'] == 'purchase']

In [53]:
# Сортируем данные по device_id и date
purchase_data = purchase_data.sort_values(by=['device_id', 'date'])

In [54]:
# Убираем дубликаты по device_id, оставляя только первую покупку
first_purchases = purchase_data.drop_duplicates(subset='device_id', keep='first')

In [55]:
# Группировка данных по каналу и расчет медианной стоимости первой покупки
median_purchase_by_channel = first_purchases.groupby('utm_source')['purchase_sum'].median()

In [56]:
# Сортировка каналов по медианной стоимости первой покупки в порядке убывания
sorted_median_purchase_by_channel = median_purchase_by_channel.sort_values(ascending=False)

In [60]:
sorted_median_purchase_by_channel.head()

utm_source
-                398.5
referal          395.5
instagram_ads    393.5
vk_ads           393.0
yandex-direct    392.5
Name: purchase_sum, dtype: float64

### Ответ: Реферальная программа